## SCALib Test

In [ ]:
# from scalib.metrics import SNR
from matplotlib import pyplot as plt
from time import perf_counter_ns
import os

from src.zarr_handler import DaskZarrHandler
from src.dask_operation import OracleAES, SNR

# import dataset
def local_dataset(name: str) -> str:
    return os.path.join(os.path.expanduser("~"), "Nextcloud", "UNH", "Research", "Projects", "SCA Datasets", name + '.zarr')
dataset_name = "SAM4S 100000 random Traces [50c]"
dset_handle = DaskZarrHandler(local_dataset(dataset_name), chunksize=2000)

# get data
k = dset_handle.get_dask_array(0, 0, "key")
ct = dset_handle.get_dask_array(0, 0, "ciphertext")
t = dset_handle.get_dask_array(0, 0, "traces")

# instantiate oracle
oracle = OracleAES(
    aes_type=128, keys=k, texts=ct, 
    round=1, step='sbox',               # finds intermediate state at round 'round' step 'step'
    lazy=False                          # lazy=False computes results immedietly on init
)

# instantiate SNR
snr = SNR(t, oracle.result)


# t0 = perf_counter_ns()
# snr.fit_u(t, oracle.result.astype(np.uint16))
# out = snr.get_snr()
# dt = perf_counter_ns() - t0

# print(f"Performed SNR calculation in {dt*1e-9:.6f} Seconds")

for i in range(snr.shape[0]):
    plt.plot(np.arange(snr.shape[1]), snr[i])
plt.show()


Module AES128 not found, compiling instrumented Tiny-AES shared library


AES128.c:988:13: warning: ‘Cipher’ defined but not used [-Wunused-function]
  988 | static void Cipher(state_t* state, const uint8_t* RoundKey)
      |             ^~~~~~


Module AES192 not found, compiling instrumented Tiny-AES shared library


AES192.c:624:9: warning: ‘AES_KEYLEN’ redefined
  624 | #define AES_KEYLEN 16   // Key length in bytes
      |         ^~~~~~~~~~
<command-line>: note: this is the location of the previous definition
AES192.c:988:13: warning: ‘Cipher’ defined but not used [-Wunused-function]
  988 | static void Cipher(state_t* state, const uint8_t* RoundKey)
      |             ^~~~~~


Module AES256 not found, compiling instrumented Tiny-AES shared library


AES256.c:624:9: warning: ‘AES_KEYLEN’ redefined
  624 | #define AES_KEYLEN 16   // Key length in bytes
      |         ^~~~~~~~~~
<command-line>: note: this is the location of the previous definition
AES256.c:988:13: warning: ‘Cipher’ defined but not used [-Wunused-function]
  988 | static void Cipher(state_t* state, const uint8_t* RoundKey)
      |             ^~~~~~


Ignoring exception:
('No input specification for OracleAES: input_conditions must be declared in OracleAES.__new__', "'OracleAES' object has no attribute 'input_conditions'")


Exception: Using SCALib from a thread without contextvars propagation is not supported.
We provide scalib.tools.ContextExecutor as a contextvars-aware replacement
for concurrent.futures.ThreadPoolExecutor.
See https://github.com/simple-crypto/SCALib/issues/89 for more information.

## Figures / data for OPTIMIST Presentation

In [ ]:
from src.ral_scarr.engines.NICV import NICV
from src.ral_scarr.engines.snr import SNR
from src.ral_scarr.file_handling.trace_handler import TraceHandler as th
from src.ral_scarr.container.container import Container, ContainerOptions
from src.ral_scarr.model_values.ciphertext import CipherText

dataset_name = "SAM4S 100000 random Traces [50c]"
handler = th(fileName=local_dataset(dataset_name), batchSize=5000)

container = Container(options=ContainerOptions(engine=engine, handler=handler), model_positions=[i for i in range(16)])
container.run()
results = np.squeeze(engine.get_result())

